<a href="https://colab.research.google.com/github/hemekci/RecommendationSystems/blob/main/AssociatonRuleRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Brief Info of Case

The data set named Online Retail II is a UK-based online sale store's sales between 01/12/2009 - 09/12/2011
The product catalog of this company includes souvenirs.
DATASET {InvoiceNo StockCode Description Quantity InvoiceDate UnitPrice (Sterling) CustomerID Country}

#Association Rule Learning Recommender (ARL)
Our goal is to apply association analysis to the online retail II dataset (in this case we will check Germany as an example) and suggest products.

1.   Data Preprocessing
2.   Preparing the ARL Data Structure (Invoice-Product Matrix)
3.   Creating Association Rules
4.   Preparing the Function of the Study
5.   Suggesting a Product to Users at the Basket Stage


# 1. Data Preprocessing

In [3]:
# 1. Data Preprocessing
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

df_ = pd.read_excel("/content/data/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()
df.head()


Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00   2.55      17850.0  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00   3.39      17850.0  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00   2.75      17850.0  United Kingdom
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00   3.39      17850.0  United Kingdom
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6 2010-12-01 08:26:00   3.39      17850.0  United Kingdom

In [7]:

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].astype(str).str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe


df = retail_data_prep(df)
# We will look only the customers from Germany
df_ge = df[df["Country"] == "Germany"]
df_ge.head()

Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID  Country
1109  536527     22809              SET OF 6 T-LIGHTS SANTA       6.0 2010-12-01 13:04:00   2.95      12662.0  Germany
1110  536527     84347  ROTATING SILVER ANGELS T-LIGHT HLDR       6.0 2010-12-01 13:04:00   2.55      12662.0  Germany
1111  536527     84945   MULTI COLOUR SILVER T-LIGHT HOLDER      12.0 2010-12-01 13:04:00   0.85      12662.0  Germany
1112  536527     22242        5 HOOK HANGER MAGIC TOADSTOOL      12.0 2010-12-01 13:04:00   1.65      12662.0  Germany
1113  536527     22244           3 HOOK HANGER MAGIC GARDEN      12.0 2010-12-01 13:04:00   1.95      12662.0  Germany

# 2. Preparing the ARL Data Structure (Invoice-Product Matrix)

In [9]:
# 2. Preparing the ARL Data Structure (Invoice-Product Matrix)
def check_invoice(dataframe, invoice):
    product_list = dataframe[dataframe["Invoice"] == 536527][["Description", "Quantity"]]
    print(product_list)

#check_invoice(df_ge,5527)


def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)
    return product_name

#check_id(df_ge, 21558)


# each invoice/product aggregate by sum
#df_ge.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).head()
df_ge.pivot_table(index=["Invoice"], columns=["Description"], values="Quantity").fillna(0)

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK   RED SPOT GIFT BAG LARGE   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  12 IVORY ROSE PEG PLACE SETTINGS  12 MESSAGE CARDS WITH ENVELOPES  12 PENCIL SMALL TUBE WOODLAND  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE SKULL  12 PENCILS TALL TUBE POSY  12 PENCILS TALL TUBE RED RETROSPOT  12 PENCILS TALL TUBE SKULLS  12 PENCILS TALL TUBE WOODLAND  12 PINK HEN+CHICKS IN BASKET  12 RED ROSE PEG PLACE SETTINGS  16 PIECE CUTLERY SET PANTRY DESIGN  2 PICTURE BOOK EGGS EASTER BUNNY  2 PICTURE BOOK EGGS EASTER CHICKS  20 DOLLY PEGS RETROSPOT  200 BENDY SKULL STRAWS  200 RED + WHITE BENDY STRAWS  3 DRAWER ANTIQUE WHITE WOOD CABINET  3 HOOK HANGER MAGIC GARDEN  3 HOOK PHOTO SHELF ANTIQUE WHITE  3 PIECE SPACEBOY COOKIE CUTTER SET  3 RAFFIA RIBBONS 50'S CHRISTMAS   3 STRIPEY MICE FELTCRAFT  3 TIER CAKE TIN GREEN AND CREAM  3 TIER CAKE TIN RED AND CREAM  3 TIER SWEETHEART GARDEN SHELF  3 TRADITIONAl BISCUIT CUTTERS  SET  36 DOILIES DOLLY GIRL  36 FOIL HEART CAKE CASES  36 FOIL STAR CAKE CASES   36 PENCILS TUBE RED RETROSPOT  36 PENCILS TUBE SKULLS  36 PENCILS TUBE WOODLAND  3D CHRISTMAS STAMPS STICKERS   3D TRADITIONAL CHRISTMAS STICKERS  3D VINTAGE CHRISTMAS STICKERS   4 LAVENDER BOTANICAL DINNER CANDLES  4 TRADITIONAL SPINNING TOPS  5 HOOK HANGER MAGIC TOADSTOOL  5 HOOK HANGER RED MAGIC TOADSTOOL  50'S CHRISTMAS PAPER GIFT BAG  6 GIFT TAGS 50'S CHRISTMAS   6 GIFT TAGS VINTAGE CHRISTMAS   6 RIBBONS ELEGANT CHRISTMAS   6 RIBBONS EMPIRE    6 RIBBONS RUSTIC CHARM  6 RIBBONS SHIMMERING PINKS   60 CAKE CASES DOLLY GIRL DESIGN  60 CAKE CASES VINTAGE CHRISTMAS  60 TEATIME FAIRY CAKE CASES  72 SWEETHEART FAIRY CAKE CASES  ABC TREASURE BOOK BOX   ACRYLIC GEOMETRIC LAMP  ACRYLIC HANGING JEWEL,PINK  ADULT APRON APPLE DELIGHT  ADVENT CALENDAR GINGHAM SACK  AIRLINE BAG VINTAGE JET SET BROWN  AIRLINE BAG VINTAGE JET SET RED  AIRLINE BAG VINTAGE JET SET WHITE  AIRLINE BAG VINTAGE TOKYO 78  AIRLINE BAG VINTAGE WORLD CHAMPION   ALARM CLOCK BAKELIKE CHOCOLATE  ALARM CLOCK BAKELIKE GREEN  ALARM CLOCK BAKELIKE IVORY  ALARM CLOCK BAKELIKE ORANGE  ALARM CLOCK BAKELIKE PINK  ALARM CLOCK BAKELIKE RED   ALPHABET HEARTS STICKER SHEET  ALPHABET STENCIL CRAFT  ANGEL DECORATION PAINTED ZINC   ANIMAL STICKERS  ANT COPPER LIME BOUDICCA BRACELET  ANT COPPER PINK BOUDICCA BRACELET  ANT COPPER TURQ BOUDICCA BRACELET  ANTIQUE GLASS DRESSING TABLE POT  ANTIQUE GLASS PEDESTAL BOWL  ANTIQUE SILVER T-LIGHT GLASS  APOTHECARY MEASURING JAR  APPLE BATH SPONGE  APRON APPLE DELIGHT  AREA PATROLLED METAL SIGN  ASS FLORAL PRINT MULTI SCREWDRIVER  ASSORTED BOTTLE TOP  MAGNETS   ASSORTED COLOUR BIRD ORNAMENT  ASSORTED COLOUR LIZARD SUCTION HOOK  ASSORTED COLOUR MINI CASES  ASSORTED COLOUR T-LIGHT HOLDER  ASSORTED COLOURS SILK FAN  ASSORTED EASTER DECORATIONS  BELLS  ASSORTED EASTER GIFT TAGS  ASSORTED FLOWER COLOUR "LEIS"  ASSORTED MONKEY SUCTION CUP HOOK  ASSORTED TUTTI FRUTTI BRACELET  ASSORTED TUTTI FRUTTI PEN  ASSTD DESIGN 3D PAPER STICKERS  ASSTD DESIGN RACING CAR PEN  ASSTD FRUIT+FLOWERS FRIDGE MAGNETS  BABUSHKA LIGHTS STRING OF 10  BABY BOOM RIBBONS   BAG 125g SWIRLY MARBLES  BAG 250g SWIRLY MARBLES  BAG 500g SWIRLY MARBLES  BAKING MOULD CHOCOLATE CUPCAKES  BAKING MOULD EASTER EGG WHITE CHOC  BAKING MOULD HEART MILK CHOCOLATE  BAKING MOULD HEART WHITE CHOCOLATE  BAKING MOULD ROSE MILK CHOCOLATE  BAKING MOULD ROSE WHITE CHOCOLATE  BAKING MOULD TOFFEE CUP CHOCOLATE  BAKING SET 9 PIECE RETROSPOT   BAKING SET SPACEBOY DESIGN  BALLOON ART MAKE YOUR OWN FLOWERS  BALLOON PUMP WITH 10 BALLOONS  BALLOON WATER BOMB PACK OF 35  BALLOONS  WRITING SET   BANQUET BIRTHDAY  CARD    BASKET OF TOADSTOOLS  BATH BUILDING BLOCK WORD  BATHROOM METAL SIGN  BEACH HUT SHELF W 3 DRAWERS  BEADED CHANDELIER T-LIGHT HOLDER  BEADED CRYSTAL HEART GREEN LARGE  BELLE JARDINIERE CUSHION COVER  BEWARE OF THE CAT METAL SIGN   BICYCLE PUNCTURE REPA

In [10]:

def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.pivot_table(index=["Invoice"], columns=["StockCode"], values="Quantity").fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.pivot_table(index=["Invoice"], columns=["Description"], values="Quantity").fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

ge_inv_pro_df = create_invoice_product_df(df_ge)


# 3. Creating Association Rules

In [17]:
# 3. Creating Association Rules

# Name of the algorithm is Apriori because it uses prior knowledge of frequent itemset properties.
# Antecedent (IF): This is an item/group of items that are typically found in the Itemsets or Datasets.
# Consequent (THEN): This comes along as an item with an Antecedent/group of Antecedents.

frequent_itemsets = apriori(ge_inv_pro_df, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False).head(50)
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.sort_values("support", ascending=False).head()
rules.sort_values("lift", ascending=False).head(50)


def create_rules(dataframe, id=True, country="Germany"):
    dataframe = dataframe[dataframe['Country'] == country]
    dataframe = create_invoice_product_df(dataframe, id)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules


rules_ge = create_rules(df, country="Germany")
rules_ge.sort_values("lift", ascending=False).head(5)




antecedents            consequents  antecedent support  consequent support   support  confidence  lift  leverage  conviction
39043         (21989, 21086)  (21987, 21988, 21094)            0.010941            0.010941  0.010941         1.0  91.4  0.010821         inf
39028  (21987, 21989, 21094)         (21988, 21086)            0.010941            0.010941  0.010941         1.0  91.4  0.010821         inf
39041         (21988, 21086)  (21987, 21989, 21094)            0.010941            0.010941  0.010941         1.0  91.4  0.010821         inf
39036         (21987, 21989)  (21086, 21988, 21094)            0.010941            0.010941  0.010941         1.0  91.4  0.010821         inf
39035         (21987, 21988)  (21086, 21989, 21094)            0.010941            0.010941  0.010941         1.0  91.4  0.010821         inf

# 4. Recommend Other Products with ARL

In [18]:
# 4. Recommend Other Products with ARL
############################################

# will offer other products for each of three products with stock code: 21987 , 23235 , 22747

product_id = 21987
check_id(df, product_id)

sorted_rules = rules.sort_values("lift", ascending=False)
recommendation_list = []


# enumarate list
# [eat'sleap, repat']
# (0, 'eat')
# (1, 'sleep')
# (2, 'repeat')

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:  # example 21987, 21987 in antedent list so take consequent's
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))


def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)  # first sort rules and later trace elements
    recommendation_list = []

    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:  # example 21987, 21987 in antedent list so take consequent's first
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))

    recommendation_list = list({item for item_list in recommendation_list for item in item_list})

    return recommendation_list[:rec_count]

arl_recommender(rules_ge, product_id , 5)

['PACK OF 6 SKULL PAPER CUPS']


['POST', 21988, 21989, 21094, 21086]

# Example : Recommend 3 products to whom buy "the products"

In [19]:
#lets recommend for the products 21987,23235, 22747 with ARL Recommender


product_ids = [21987, 23235, 22747]
arl_recommender(rules_ge, 21987, 3)
rules_ge = create_rules(df_ge, country="Germany")

#arl_recommender(rules_ge, product_ids[0], 3)


for pid in product_ids:
    product_name = check_id(df_ge, pid)
    my_recommendation_list = arl_recommender(rules_ge, pid, 3)
    listToStr = ' '.join(map(str, my_recommendation_list))
    print(str(pid) + "- 3 id recommendations - " + listToStr + "\n")



['PACK OF 6 SKULL PAPER CUPS']
21987- 3 id recommendations - POST 21988 21989

['STORAGE TIN VINTAGE LEAF']
23235- 3 id recommendations - POST 23236 23237

["POPPY'S PLAYHOUSE BATHROOM"]
22747- 3 id recommendations - 22745 22746 22748

